In [ ]:
import sys
sys.path.append('/root/alok/github/aquabyte_biomass')
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
import statsmodels.api as sm
from biomass_utils.points_of_interest import get_data, get_depth_cutoff, get_points_of_interest, distance_between_points

In [ ]:
data_dir_base = '/root/data/blender_v3'
idx = 0

data = get_data(data_dir_base, idx)
point_cloud, mask, annotation, image = data['point_cloud'], data['mask'], data['annotation_data'], data['image']

#---------- THIS IS TEMPORARY -- CHANGE ONCE BLENDER DATA IS CORRECTED!!-------#
x = annotation['sensor_height'] 
annotation['sensor_height'] = annotation['sensor_width']
annotation['sensor_width'] = x
#------------------------------------------------------------------------------#

depth_map = point_cloud[:,:,1]
cutoff = get_depth_cutoff(depth_map, mask)
poi = get_points_of_interest(mask, depth_map, cutoff)

<h1> Perform Volume Integration Technique </h1>

In [ ]:
length_endpoint_1_world = point_cloud[poi['length_endpoint_1']]
length_endpoint_2_world = point_cloud[poi['length_endpoint_2']]
width_endpoint_1_world = point_cloud[poi['width_endpoint_1']]
width_endpoint_2_world = point_cloud[poi['width_endpoint_2']]
visible_centroid_world = point_cloud[poi['visible_centroid']]
a = np.vstack([np.take(length_endpoint_1_world, [0, 2]), 
               np.take(length_endpoint_2_world, [0, 2]), 
               np.take(width_endpoint_1_world, [0, 2]),
               np.take(width_endpoint_2_world, [0, 2])])
y = np.array([length_endpoint_1_world[1], length_endpoint_2_world[1], width_endpoint_1_world[1], width_endpoint_2_world[1]])
A = np.c_[a, np.ones(a.shape[0])]
res = np.linalg.lstsq(A, y)
A, C, D = res[0]
B = -1

distance_to_plane_matrix = np.abs((np.dot(point_cloud, np.array([A, B, C])) + D))/(np.sqrt(A**2 + B**2 + C**2))
surface_distances = distance_to_plane_matrix * (mask > 0) * (depth_map < cutoff)
delta_x = (annotation['sensor_width'] * depth_map)/(mask.shape[1] * annotation['focal_length'])
delta_z = (annotation['sensor_height'] * depth_map)/(mask.shape[0] * annotation['focal_length'])
volume = (delta_x * delta_z * surface_distances).sum() * 2
print(volume)





In [ ]:
draw = ImageDraw.Draw(image)
draw.line((poi['length_endpoint_1'][1], poi['length_endpoint_1'][0], poi['length_endpoint_2'][1], poi['length_endpoint_2'][0]), fill=255)
draw.line((poi['width_endpoint_1'][1], poi['width_endpoint_1'][0], poi['width_endpoint_2'][1], poi['width_endpoint_2'][0]), fill=255)
image

In [ ]:
COMPLETE_SAMPLE_SIZE = 4007
df = pd.DataFrame()
for idx in range(COMPLETE_SAMPLE_SIZE):
    if idx % 10 == 0:
        print(idx)
        
    data = get_data(data_dir_base, idx)
    point_cloud, mask, annotation, image = data['point_cloud'], data['mask'], data['annotation_data'], data['image']

    #---------- THIS IS TEMPORARY -- CHANGE ONCE BLENDER DATA IS CORRECTED!!-------#
    x = annotation['sensor_height'] 
    annotation['sensor_height'] = annotation['sensor_width']
    annotation['sensor_width'] = x
    #------------------------------------------------------------------------------#

    depth_map = point_cloud[:,:,1]
    cutoff = get_depth_cutoff(depth_map, mask)
    poi = get_points_of_interest(mask, depth_map, cutoff)    
    
    length_endpoint_1_world = point_cloud[poi['length_endpoint_1']]
    length_endpoint_2_world = point_cloud[poi['length_endpoint_2']]
    width_endpoint_1_world = point_cloud[poi['width_endpoint_1']]
    width_endpoint_2_world = point_cloud[poi['width_endpoint_2']]
    visible_centroid_world = point_cloud[poi['visible_centroid']]
    
    # get length and width
    length = distance_between_points(length_endpoint_1_world, length_endpoint_2_world)
    width = distance_between_points(width_endpoint_1_world, width_endpoint_2_world)
    
    # get volume via integration method
    a = np.vstack([np.take(length_endpoint_1_world, [0, 2]), 
                   np.take(length_endpoint_2_world, [0, 2]), 
                   np.take(width_endpoint_1_world, [0, 2]),
                   np.take(width_endpoint_2_world, [0, 2])])
    y = np.array([length_endpoint_1_world[1], length_endpoint_2_world[1], width_endpoint_1_world[1], width_endpoint_2_world[1]])
    A = np.c_[a, np.ones(a.shape[0])]
    res = np.linalg.lstsq(A, y)
    A, C, D = res[0]
    B = -1

    distance_to_plane_matrix = np.abs((np.dot(point_cloud, np.array([A, B, C])) + D))/(np.sqrt(A**2 + B**2 + C**2))
    surface_distances = distance_to_plane_matrix * (mask > 0) * (depth_map < cutoff)
    delta_x = (annotation['sensor_width'] * depth_map)/(mask.shape[1] * annotation['focal_length'])
    delta_z = (annotation['sensor_height'] * depth_map)/(mask.shape[0] * annotation['focal_length'])
    volume = (delta_x * delta_z * surface_distances).sum() * 2

    ground_truth_length = annotation['length']
    ground_truth_width = annotation['height']
    ground_truth_breadth = annotation['width']
    ground_truth_volume = annotation['volume']

    line_to_append = { 
        'predicted_length': length, 
        'predicted_width': width,
        'predicted_volume': volume,
        'ground_truth_length': ground_truth_length,
        'ground_truth_width': ground_truth_width,
        'ground_truth_breadth': ground_truth_breadth,
        'ground_truth_volume': ground_truth_volume
    }
    df = df.append(line_to_append, ignore_index=True)
    

In [ ]:
df

In [ ]:
features = ['predicted_volume']
target = 'ground_truth_volume'

errors = []
for i in range(100):
    in_sample_mask = df.index.isin(df.sample(2000).index)
    model = sm.QuantReg(df.ix[in_sample_mask, target], df.ix[in_sample_mask, features]).fit(q=0.5)

    coefficient = model.params.predicted_volume
    df['prediction'] = 1.3119 * df.predicted_volume
    outlier_removal_mask = ((df.prediction / df.ground_truth_volume) > 0.5) & ((df.prediction / df.ground_truth_volume) < 2)
    avg_prediction = df.ix[~in_sample_mask & outlier_removal_mask, 'prediction'].mean()
    avg_ground_truth = df.ix[~in_sample_mask & outlier_removal_mask, 'ground_truth_volume'].mean()
    error = abs(avg_prediction - avg_ground_truth) / avg_ground_truth
    errors.append(100*error)
    

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(errors)
plt.xlabel('Percentage error of predicted average biomass vs. ground truth average biomass')
plt.ylabel('Count')

In [ ]:
((df.ix[~in_sample_mask, 'ground_truth_volume'] - 1.2914*(df.ix[~in_sample_mask, 'predicted_volume']))/(1.2914*(df.ix[~in_sample_mask, 'predicted_volume']))).mean()

In [ ]:
plt.figure(figsize=(15, 10))

plt.scatter(df.predicted_volume, df.ground_truth_volume)
plt.plot([range_min, range_max], [1.3029*range_min, 1.3029*range_max])
plt.show()